In [24]:
####### Good Translator(Quiz)  ############

import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk
from deep_translator import GoogleTranslator
from gtts import gTTS
from playsound import playsound
import speech_recognition as sr
import datetime
import csv
import os
import json
import random
import time

# -------------------------
# Splash screen
# -------------------------
def show_splash(root, callback):
    splash = tk.Toplevel(root)
    splash.title("Welcome")
    splash.geometry("1000x650")
    splash.configure(bg="#ffffff")
    splash.overrideredirect(True)

    try:
        img = Image.open("translator_logo.jpg")
        img = img.resize((1000, 650))
        img_tk = ImageTk.PhotoImage(img)
        label = tk.Label(splash, image=img_tk)
        label.image = img_tk
        label.place(x=0, y=0)
    except Exception:
        label = tk.Label(splash, text="🌐 TransFlip App", font=("Helvetica", 30), bg="#ffffff")
        label.pack(expand=True)

    typing_label = tk.Label(splash, text="", font=("Helvetica", 20, "italic"), bg="#ffffff", fg="#1a1a1a")
    typing_label.place(relx=0.5, rely=0.85, anchor="center")

    def type_text(text, i=0):
        if i < len(text):
            typing_label.config(text=text[:i+1])
            splash.after(100, lambda: type_text(text, i+1))
    type_text("Starting Real TransFlip App...")

    progress = ttk.Progressbar(splash, orient="horizontal", length=400, mode="determinate")
    progress.place(relx=0.5, rely=0.93, anchor="center")

    def load_bar(value=0):
        if value <= 100:
            progress["value"] = value
            splash.after(30, lambda: load_bar(value + 2))
        else:
            splash.destroy()
            callback()

    splash.attributes("-alpha", 0.0)
    def fade_in(alpha=0.0):
        if alpha < 1.0:
            splash.attributes("-alpha", alpha)
            splash.after(50, lambda: fade_in(alpha + 0.05))
        else:
            load_bar()
    fade_in()


# -------------------------
# Config / Globals
# -------------------------
HISTORY_CACHE_FILE = "transflip_history.json"
CACHE_MAX = 200
QUIZ_PROGRESS_FILE = "quiz_progress.json"

LANGS = {
    "Auto-Detect": "auto",
    "English": "en",
    "Hindi": "hi",
    "Marathi": "mr",
    "Spanish": "es",
    "French": "fr",
    "German": "de",
    "Italian": "it",
    "Chinese (Simplified)": "zh-CN",
    "Japanese": "ja",
    "Korean": "ko",
    "Russian": "ru",
    "Arabic": "ar",
    "Portuguese": "pt",
    "Bengali": "bn",
    "Tamil": "ta",
    "Telugu": "te",
    "Gujarati": "gu",
    "Urdu": "ur",
    "Punjabi": "pa",
}

translation_history = []
user_points = 0
user_streak = 0

QUIZ_POOL = ["Hello", "Thank you", "Good morning", "How are you?", "Friend", "School", "Love", "Peace",
             "Food", "Water", "Teacher", "Home", "Book", "Computer", "Sun", "Moon", "Happy", "Sad"]

# -------------------------
# Utilities
# -------------------------
def now_str():
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


def load_history_cache():
    global translation_history
    if os.path.exists(HISTORY_CACHE_FILE):
        try:
            with open(HISTORY_CACHE_FILE, "r", encoding="utf-8") as f:
                translation_history = json.load(f)
        except Exception:
            translation_history = []
    else:
        translation_history = []


def save_history_cache():
    try:
        with open(HISTORY_CACHE_FILE, "w", encoding="utf-8") as f:
            json.dump(translation_history[-CACHE_MAX:], f, ensure_ascii=False, indent=2)
    except Exception:
        pass


def push_history(original, translated, src, tgt):
    global translation_history
    item = {
        "id": str(len(translation_history) + 1) + "_" + str(int(datetime.datetime.now().timestamp())),
        "original": original,
        "translated": translated,
        "src": src,
        "tgt": tgt,
        "time": now_str(),
        "favorite": False,
    }
    translation_history.insert(0, item)
    save_history_cache()
    return item


# -------------------------
# Main App Class
# -------------------------
class TransFlipApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🔄 TransFlip — Enhanced")
        self.root.geometry("1100x720")
        self.root.minsize(980, 640)

        # theme state
        self.dark_mode = tk.BooleanVar(value=False)
        self.tts_slow = tk.BooleanVar(value=False)

        # settings
        self.pref_lang = tk.StringVar(value="English")
        self.fun_mode = tk.BooleanVar(value=False)
        self.clear_before_speech = tk.BooleanVar(value=True)

        # status bar var
        self.status_var = tk.StringVar(value="Ready")

        # quiz internal
        self.quiz_phrase_var = tk.StringVar(value=random.choice(QUIZ_POOL))

        # load history
        load_history_cache()

        # build UI
        self.build_splashless_ui()

    # -------------------------
    # UI build (Notebook tabs + header + status bar)
    # -------------------------
    def build_splashless_ui(self):
        header = tk.Frame(self.root, bg="#0078D7", height=56)
        header.pack(fill="x")
        header.pack_propagate(False)
        tk.Label(header, text="🔄 TransFlip", fg="white", bg="#0078D7",
                 font=("Helvetica", 16, "bold")).pack(side="left", padx=12)

        tk.Label(header, text="Fast. Simple. Friendly.", fg="#cfe8ff", bg="#0078D7",
                 font=("Helvetica", 9)).pack(side="left", padx=6)

        ttk.Checkbutton(header, text="🌙 Dark Mode", variable=self.dark_mode,
                        command=self.apply_theme).pack(side="right", padx=12)

        style = ttk.Style(self.root)
        try:
            style.theme_use("clam")
        except Exception:
            pass
        style.configure("TNotebook.Tab", padding=[12, 8], font=("Helvetica", 12, "bold"))

        nb = ttk.Notebook(self.root)
        nb.pack(fill="both", expand=True, padx=8, pady=8)

        self.tab_translator = ttk.Frame(nb)
        self.tab_history = ttk.Frame(nb)
        self.tab_quiz = ttk.Frame(nb)
        self.tab_personal = ttk.Frame(nb)

        nb.add(self.tab_translator, text="Translator")
        nb.add(self.tab_history, text="History")
        nb.add(self.tab_quiz, text="Quiz")
        nb.add(self.tab_personal, text="Personalization")

        self.build_tab_translator(self.tab_translator)
        self.build_tab_history(self.tab_history)
        self.build_tab_quiz(self.tab_quiz)
        self.build_tab_personal(self.tab_personal)

        status_bar = tk.Label(self.root, textvariable=self.status_var,
                              bd=1, relief="sunken", anchor="w", bg="#f1f1f1")
        status_bar.pack(side="bottom", fill="x")

        self.apply_theme()

    # -------------------------
    # TAB: Translator
    # -------------------------
    def build_tab_translator(self, parent):
        container = tk.Frame(parent, bg="#f8f9fa")
        container.pack(fill="both", expand=True, padx=12, pady=12)

        topbar = tk.Frame(container, bg="#f8f9fa")
        topbar.pack(fill="x", pady=(0, 8))

        tk.Label(topbar, text="From:", bg="#f8f9fa").pack(side="left", padx=6)
        self.cb_from = ttk.Combobox(topbar, values=list(LANGS.keys()), state="readonly", width=18)
        self.cb_from.set("Auto-Detect")
        self.cb_from.pack(side="left", padx=6)

        swap_btn = tk.Button(topbar, text="🔄 Swap", command=self.swap_langs)
        swap_btn.pack(side="left", padx=6)
        self._add_hover(swap_btn)

        tk.Label(topbar, text="To:", bg="#f8f9fa").pack(side="left", padx=6)
        self.cb_to = ttk.Combobox(topbar, values=list(LANGS.keys()), state="readonly", width=18)
        self.cb_to.set("Hindi")
        self.cb_to.pack(side="left", padx=6)

        self.auto_speak = tk.BooleanVar(value=False)
        ttk.Checkbutton(topbar, text="🔊 Auto Speak", variable=self.auto_speak).pack(side="left", padx=8)

        io_frame = tk.Frame(container, bg="#f8f9fa")
        io_frame.pack(fill="both", expand=True)

        in_card = tk.LabelFrame(io_frame, text="Input", font=("Helvetica", 11, "bold"),
                                bg="white", bd=2, relief="groove")
        in_card.grid(row=0, column=0, padx=8, pady=8, sticky="nsew")
        self.input_text = tk.Text(in_card, width=60, height=12, font=("Consolas", 12), wrap="word")
        self.input_text.pack(fill="both", expand=True, padx=6, pady=6)

        out_card = tk.LabelFrame(io_frame, text="Output", font=("Helvetica", 11, "bold"),
                                 bg="white", bd=2, relief="groove")
        out_card.grid(row=0, column=1, padx=8, pady=8, sticky="nsew")
        self.output_text = tk.Text(out_card, width=60, height=12, font=("Consolas", 12), wrap="word", fg="green")
        self.output_text.pack(fill="both", expand=True, padx=6, pady=6)

        io_frame.columnconfigure(0, weight=1)
        io_frame.columnconfigure(1, weight=1)

        btn_frame = tk.Frame(container, bg="#f8f9fa")
        btn_frame.pack(pady=8)

        b_speak = tk.Button(btn_frame, text="🎤 Speak (Clear)", command=self.speech_to_text_clear)
        b_translate = tk.Button(btn_frame, text="🔁 Translate", command=self.translate_action)
        b_tts = tk.Button(btn_frame, text="🔊 Speak Output", command=self.speak_output)
        b_copy = tk.Button(btn_frame, text="📋 Copy Output", command=self.copy_output)
        b_save = tk.Button(btn_frame, text="💾 Save MP3", command=self.save_mp3)
        b_history = tk.Button(btn_frame, text="📜 Open History", command=lambda: self.open_tab_history())

        for i, b in enumerate((b_speak, b_translate, b_tts, b_copy, b_save, b_history)):
            b.grid(row=0, column=i, padx=6, ipadx=6, ipady=4)
            self._add_hover(b)

    # -------------------------
    # TAB: History
    # -------------------------
    def build_tab_history(self, parent):
        container = tk.Frame(parent, bg="#f8f9fa")
        container.pack(fill="both", expand=True, padx=12, pady=12)

        hist_card = tk.LabelFrame(container, text="Translation History", font=("Helvetica", 11, "bold"),
                                  bg="white", bd=2, relief="groove")
        hist_card.pack(fill="both", expand=True, padx=8, pady=8)

        toolrow = tk.Frame(hist_card, bg="white")
        toolrow.pack(fill="x", pady=(4, 8))

        tk.Label(toolrow, text="Search:", bg="white").pack(side="left", padx=4)
        self.search_var = tk.StringVar()
        search_entry = ttk.Entry(toolrow, textvariable=self.search_var, width=36)
        search_entry.pack(side="left", padx=4)
        search_entry.bind("<KeyRelease>", lambda e: self.refresh_history_tree())

        self.fav_only_var = tk.BooleanVar(value=False)
        ttk.Checkbutton(toolrow, text="★ Favorites only", variable=self.fav_only_var,
                        command=self.refresh_history_tree).pack(side="left", padx=8)

        ttk.Button(toolrow, text="🔄 Refresh", command=self.refresh_history_tree).pack(side="left", padx=6)
        ttk.Button(toolrow, text="🗑️ Delete Selected", command=self.delete_selected_history).pack(side="left", padx=6)
        ttk.Button(toolrow, text="📋 Copy Selected", command=self.copy_selected_history).pack(side="left", padx=6)
        ttk.Button(toolrow, text="📤 Export", command=self.export_history).pack(side="left", padx=6)
        ttk.Button(toolrow, text="⭐ Toggle Favorite", command=self.toggle_favorite_selected).pack(side="left", padx=6)

        cols = ("orig", "trans", "time", "fav")
        self.tree = ttk.Treeview(hist_card, columns=cols, show="headings", height=14)
        self.tree.heading("orig", text="Original")
        self.tree.heading("trans", text="Translated")
        self.tree.heading("time", text="Timestamp")
        self.tree.heading("fav", text="★")
        self.tree.column("orig", width=360, anchor="w")
        self.tree.column("trans", width=360, anchor="w")
        self.tree.column("time", width=140, anchor="center")
        self.tree.column("fav", width=40, anchor="center")

        vsb = ttk.Scrollbar(hist_card, orient="vertical", command=self.tree.yview)
        hsb = ttk.Scrollbar(hist_card, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscroll=vsb.set, xscroll=hsb.set)
        self.tree.pack(side="left", fill="both", expand=True)
        vsb.pack(side="left", fill="y")
        hsb.pack(side="bottom", fill="x")

        self.tree.bind("<Double-1>", self.on_tree_double_click)

        self.refresh_history_tree()

    # -------------------------
    # TAB: Quiz (ENHANCED)
    # -------------------------
    def build_tab_quiz(self, parent):
        container = tk.Frame(parent, bg="#f8f9fa")
        container.pack(fill="both", expand=True, padx=12, pady=12)

        quiz_card = tk.LabelFrame(container, text="Fun Learning Quiz", font=("Helvetica", 11, "bold"),
                                  bg="white", bd=2, relief="groove")
        quiz_card.pack(fill="both", expand=True, padx=8, pady=8)

        # Top row: language selector + mode
        toprow = tk.Frame(quiz_card, bg="white")
        toprow.pack(fill="x", pady=6, padx=8)

        tk.Label(toprow, text="Quiz language (translate TO):", bg="white").pack(side="left", padx=(4,6))
        self.quiz_lang_cb = ttk.Combobox(toprow, values=list(LANGS.keys()), state="readonly", width=18)
        try:
            self.quiz_lang_cb.set(self.cb_to.get())
        except Exception:
            self.quiz_lang_cb.set("Hindi")
        self.quiz_lang_cb.pack(side="left", padx=4)

        ttk.Separator(toprow, orient="vertical").pack(side="left", padx=8, fill="y")

        tk.Label(toprow, text="Mode:", bg="white").pack(side="left", padx=(6,4))
        self.quiz_mode = tk.StringVar(value="MCQ")  # MCQ or Text
        ttk.Radiobutton(toprow, text="MCQ", variable=self.quiz_mode, value="MCQ").pack(side="left", padx=4)
        ttk.Radiobutton(toprow, text="Text", variable=self.quiz_mode, value="Text").pack(side="left", padx=4)

        # Question label
        self.quiz_q_label = tk.Label(quiz_card, textvariable=self.quiz_phrase_var,
                                     font=("Helvetica", 16, "bold"), bg="white")
        self.quiz_q_label.pack(pady=8)

        # Answer area: MCQ buttons
        self.mcq_frame = tk.Frame(quiz_card, bg="white")
        self.mcq_frame.pack(fill="x", pady=6)

        self.mcq_buttons = []
        for i in range(4):
            b = tk.Button(self.mcq_frame, text=f"Option {i+1}", width=36, wraplength=300,
                          command=lambda idx=i: self._on_mcq_select(idx))
            b.grid(row=i//2, column=i%2, padx=8, pady=8)
            self._add_hover(b)
            self.mcq_buttons.append(b)

        # Text entry fallback
        self.quiz_answer_entry = ttk.Entry(quiz_card, width=60)
        self.quiz_answer_entry.pack(pady=6)

        # Buttons row: New, Check, Speak Q, Speak Expected, Record Answer
        btn_frame = tk.Frame(quiz_card, bg="white")
        btn_frame.pack(pady=6)

        ttk.Button(btn_frame, text="New Question", command=self.quiz_new_question).grid(row=0, column=0, padx=6)
        ttk.Button(btn_frame, text="Check Answer", command=self.check_quiz_answer).grid(row=0, column=1, padx=6)
        ttk.Button(btn_frame, text="🔊 Speak Question", command=lambda: self.speak_text(self.quiz_q_label.cget("text"), use_quiz_lang=True)).grid(row=0, column=2, padx=6)
        ttk.Button(btn_frame, text="🔊 Speak Expected", command=self.speak_expected_answer).grid(row=0, column=3, padx=6)
        ttk.Button(btn_frame, text="🎤 Speak Answer (Record)", command=self.record_answer_and_fill).grid(row=0, column=4, padx=6)
        
        # Result label
        self.quiz_result_label = tk.Label(quiz_card, text="", font=("Helvetica", 12), bg="white")
        self.quiz_result_label.pack(pady=8)

        # Progress & stats row
        stats_frame = tk.Frame(quiz_card, bg="white")
        stats_frame.pack(fill="x", padx=12, pady=6)
        self.quiz_progress = ttk.Progressbar(stats_frame, orient="horizontal", length=300, mode="determinate")
        self.quiz_progress.pack(side="left", padx=(0,12))
        self.quiz_stats_label = tk.Label(stats_frame, text=self._stats_text(), bg="white")
        self.quiz_stats_label.pack(side="left")

        # Load saved progress
        self._load_quiz_progress()

        # initialize question and widgets
        self.quiz_new_question()
        self._update_quiz_mode_widgets()
        self.quiz_mode.trace_add("write", lambda *a: self._update_quiz_mode_widgets())

    def quiz_new_question(self):
        phrase = random.choice(QUIZ_POOL)
        self.quiz_phrase_var.set(phrase)
        try:
            self.quiz_answer_entry.delete(0, tk.END)
        except Exception:
            pass
        self.quiz_result_label.config(text="")
        tgt_label = self.quiz_lang_cb.get() if hasattr(self, "quiz_lang_cb") else "Hindi"
        tgt_code = LANGS.get(tgt_label, "hi")
        try:
            expected = GoogleTranslator(source="en", target=tgt_code).translate(phrase)
        except Exception:
            expected = "[translation error]"
        self._current_expected = expected

        # prepare MCQ options
        opts = [expected]
        attempts = 0
        while len(opts) < 4 and attempts < 20:
            w = random.choice(QUIZ_POOL)
            if w == phrase:
                attempts += 1
                continue
            try:
                tr = GoogleTranslator(source="en", target=tgt_code).translate(w)
            except Exception:
                tr = w
            if tr not in opts:
                opts.append(tr)
            attempts += 1
        random.shuffle(opts)
        for i, b in enumerate(self.mcq_buttons):
            try:
                b.config(text=opts[i], state="normal")
            except Exception:
                b.config(text="", state="disabled")

        # update progress bar and stats label
        level = self._quiz_progress.get("level", 1)
        level_threshold = level * 100
        self.quiz_progress["maximum"] = level_threshold
        self.quiz_progress["value"] = min(self._quiz_progress.get("points", 0), level_threshold)
        self.quiz_stats_label.config(text=self._stats_text())
        # clear any previous mcq selection
        self._mcq_selected_text = ""

    def check_quiz_answer(self):
        mode = self.quiz_mode.get()
        user_ans = ""
        if mode == "MCQ":
            user_ans = getattr(self, "_mcq_selected_text", "").strip()
        else:
            user_ans = self.quiz_answer_entry.get().strip() if hasattr(self, "quiz_answer_entry") else ""

        if not user_ans:
            messagebox.showinfo("Quiz", "Please choose or enter an answer.")
            return

        def norm(s): return "".join(ch for ch in s.lower().strip() if ch.isalnum() or ch.isspace())

        expected = getattr(self, "_current_expected", "")
        if not expected:
            messagebox.showerror("Quiz", "Expected answer missing. Try 'New Question'.")
            return

        correct = norm(user_ans) == norm(expected)
        global user_points, user_streak
        if correct:
            user_points += 10
            user_streak += 1
            self._quiz_progress["points"] = self._quiz_progress.get("points", 0) + 10
            level = self._quiz_progress.get("level", 1)
            if self._quiz_progress["points"] >= level * 100:
                self._quiz_progress["level"] = level + 1
                badge = self._award_badge_for_level(level + 1)
                msg = f"✅ Correct — {expected}\nLeveled up to {level+1}! Badge: {badge}"
                self.quiz_result_label.config(text=msg, fg="green")
                self.speak_text(f"Correct! Level {level+1}. Badge: {badge}", use_quiz_lang=False)
            else:
                self.quiz_result_label.config(text=f"✅ Correct — {expected}\nPoints: {user_points} | Streak: {user_streak}", fg="green")
                self.speak_text("Correct!", use_quiz_lang=False)
        else:
            user_streak = 0
            self._quiz_progress["points"] = max(0, self._quiz_progress.get("points", 0) - 5)
            self.quiz_result_label.config(text=f"❌ Not exact — Expected: {expected}", fg="red")
            self.speak_text("Incorrect", use_quiz_lang=False)

        self._save_quiz_progress()
        level = self._quiz_progress.get("level", 1)
        level_threshold = level * 100
        self.quiz_progress["maximum"] = level_threshold
        self.quiz_progress["value"] = min(self._quiz_progress.get("points", 0), level_threshold)
        self.quiz_stats_label.config(text=self._stats_text())

        
    # -------------------------
    # TAB: Personalization
    # -------------------------
    def build_tab_personal(self, parent):
        container = tk.Frame(parent, bg="#f8f9fa")
        container.pack(fill="both", expand=True, padx=12, pady=12)

        pers_card = tk.LabelFrame(container, text="Customize Experience", font=("Helvetica", 11, "bold"),
                                  bg="white", bd=2, relief="groove")
        pers_card.pack(fill="both", expand=True, padx=8, pady=8)

        tk.Label(pers_card, text="Preferred Language:", font=("Helvetica", 11), bg="white").pack(anchor="w", padx=12, pady=6)
        self.pref_lang_cb = ttk.Combobox(pers_card, values=list(LANGS.keys()), state="readonly", width=22, textvariable=self.pref_lang)
        self.pref_lang_cb.set(self.pref_lang.get())
        self.pref_lang_cb.pack(anchor="w", padx=12, pady=6)

        tk.Label(pers_card, text="Fun Learning Mode:", font=("Helvetica", 11), bg="white").pack(anchor="w", padx=12, pady=6)
        ttk.Checkbutton(pers_card, text="Enable Challenges", variable=self.fun_mode).pack(anchor="w", padx=12, pady=4)

        ttk.Checkbutton(pers_card, text="Use slow TTS (slower pronunciation)", variable=self.tts_slow).pack(anchor="w", padx=12, pady=4)
        ttk.Checkbutton(pers_card, text="Clear input before capturing speech", variable=self.clear_before_speech).pack(anchor="w", padx=12, pady=4)

        btnrow = tk.Frame(pers_card, bg="white")
        btnrow.pack(pady=10, anchor="w")
        ttk.Button(btnrow, text="Clear History Cache", command=self.clear_history_cache).pack(side="left", padx=6)
        ttk.Button(btnrow, text="Export All History", command=self.export_history).pack(side="left", padx=6)

        self.stats_label = tk.Label(pers_card, text=self._stats_text(), justify="left", bg="white")
        self.stats_label.pack(anchor="w", pady=10)

        save_btn = tk.Button(container, text="💾 Save Settings", command=self.save_settings)
        save_btn.pack(pady=12)
        self._add_hover(save_btn)

    # -------------------------
    # Actions: Translator
    # -------------------------
    def swap_langs(self):
        a = self.cb_from.get()
        b = self.cb_to.get()
        self.cb_from.set(b if b else "Auto-Detect")
        self.cb_to.set(a if a else "Hindi")

    def speech_to_text_clear(self):
        if self.clear_before_speech.get():
            self.input_text.delete("1.0", tk.END)
        recognizer = sr.Recognizer()
        try:
            with sr.Microphone() as source:
                self._status("Listening… speak now")
                audio = recognizer.listen(source, timeout=6, phrase_time_limit=12)
                try:
                    text = recognizer.recognize_google(audio)
                    self.input_text.insert(tk.END, text)
                    self._status("Captured speech")
                except sr.UnknownValueError:
                    self._status("Could not understand")
                    messagebox.showinfo("STT", "Could not understand audio")
                except sr.RequestError:
                    self._status("Network error")
                    messagebox.showerror("STT", "Network error during speech recognition")
        except Exception as e:
            messagebox.showerror("Microphone Error", str(e))
            self._status("Microphone error")

    def translate_action(self):
        src_label = self.cb_from.get()
        tgt_label = self.cb_to.get()
        src = LANGS.get(src_label, "auto")
        tgt = LANGS.get(tgt_label, "hi")
        original = self.input_text.get("1.0", tk.END).strip()
        if not original:
            return
        try:
            translated = GoogleTranslator(source=src, target=tgt).translate(original)
        except Exception as e:
            translated = f"[Error: {e}]"

        self.output_text.delete("1.0", tk.END)
        self.output_text.insert(tk.END, translated)
        item = push_history(original, translated, src, tgt)
        self.refresh_history_tree()
        self._status("Translated ✓")
        if self.auto_speak.get():
            self.speak_output()
        try:
            self.stats_label.config(text=self._stats_text())
        except Exception:
            pass

    def speak_output(self):
        text = self.output_text.get("1.0", tk.END).strip()
        if not text:
            return
        try:
            slow = self.tts_slow.get()
            tts = gTTS(text=text, lang=LANGS.get(self.cb_to.get(), "hi"), slow=slow)
            fname = "temp_tts.mp3"
            tts.save(fname)
            playsound(fname)
            try:
                os.remove(fname)
            except Exception:
                pass
        except Exception as e:
            messagebox.showerror("TTS Error", str(e))

    def copy_output(self):
        txt = self.output_text.get("1.0", tk.END).strip()
        if txt:
            self.root.clipboard_clear()
            self.root.clipboard_append(txt)
            self._status("Copied output to clipboard")

    def save_mp3(self):
        text = self.output_text.get("1.0", tk.END).strip()
        if not text:
            messagebox.showinfo("Save MP3", "No translated text to save.")
            return
        try:
            slow = self.tts_slow.get()
            tts = gTTS(text=text, lang=LANGS.get(self.cb_to.get(), "hi"), slow=slow)
            fpath = filedialog.asksaveasfilename(defaultextension=".mp3",
                                                 filetypes=[("MP3 files", "*.mp3")], initialfile="translation.mp3")
            if fpath:
                tts.save(fpath)
                messagebox.showinfo("Saved", f"Saved to {fpath}")
        except Exception as e:
            messagebox.showerror("Save MP3 Error", str(e))

    # -------------------------
    # Actions: History / Tree
    # -------------------------
    def refresh_history_tree(self):
        try:
            for r in self.tree.get_children():
                self.tree.delete(r)
        except Exception:
            pass

        q = self.search_var.get().strip().lower() if hasattr(self, "search_var") else ""
        fav_only = self.fav_only_var.get() if hasattr(self, "fav_only_var") else False
        for idx, item in enumerate(translation_history):
            if fav_only and not item.get("favorite", False):
                continue
            combined = (item["original"] + " " + item["translated"]).lower()
            if q and q not in combined:
                continue
            fav_mark = "★" if item.get("favorite", False) else ""
            try:
                self.tree.insert("", "end", iid=item["id"], values=(item["original"], item["translated"], item["time"], fav_mark))
            except Exception:
                pass

    def on_tree_double_click(self, event):
        sel = self.tree.focus()
        if not sel:
            return
        item = next((i for i in translation_history if i["id"] == sel), None)
        if item:
            self.input_text.delete("1.0", tk.END)
            self.input_text.insert("1.0", item["original"])
            self.open_tab_translator()

    def delete_selected_history(self):
        sel = self.tree.focus()
        if not sel:
            messagebox.showinfo("Delete", "Select a history row first (single-click).")
            return
        global translation_history
        translation_history = [i for i in translation_history if i["id"] != sel]
        save_history_cache()
        self.refresh_history_tree()
        self._status("Deleted selected history")
        try:
            self.stats_label.config(text=self._stats_text())
        except Exception:
            pass

    def copy_selected_history(self):
        sel = self.tree.focus()
        if not sel:
            messagebox.showinfo("Copy", "Select a history row first.")
            return
        item = next((i for i in translation_history if i["id"] == sel), None)
        if item:
            txt = f"{item['original']} → {item['translated']}"
            self.root.clipboard_clear()
            self.root.clipboard_append(txt)
            self._status("Copied history to clipboard")

    def export_history(self):
        if not translation_history:
            messagebox.showinfo("Export", "No history to export.")
            return
        fpath = filedialog.asksaveasfilename(defaultextension=".csv",
                                             filetypes=[("CSV", "*.csv"), ("Text", "*.txt")], initialfile="transflip_history.csv")
        if not fpath:
            return
        try:
            if fpath.lower().endswith(".csv"):
                with open(fpath, "w", newline='', encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow(["favorite", "original", "translated", "src", "tgt", "time"])
                    for item in translation_history:
                        writer.writerow(["yes" if item.get("favorite", False) else "no", item["original"], item["translated"],
                                         item["src"], item["tgt"], item["time"]])
            else:
                with open(fpath, "w", encoding="utf-8") as f:
                    for item in translation_history:
                        f.write(f"{'[★]' if item.get('favorite', False) else ''}{item['time']} {item['original']} → {item['translated']}\n")
            messagebox.showinfo("Export", f"Exported to {fpath}")
        except Exception as e:
            messagebox.showerror("Export Error", str(e))

    def toggle_favorite_selected(self):
        sel = self.tree.focus()
        if not sel:
            messagebox.showinfo("Favorite", "Select a history row first.")
            return
        item = next((i for i in translation_history if i["id"] == sel), None)
        if item:
            item["favorite"] = not item.get("favorite", False)
            translation_history.sort(key=lambda it: (not it.get("favorite", False), it.get("time", "")), reverse=False)
            save_history_cache()
            self.refresh_history_tree()
            self._status("Toggled favorite")
            try:
                self.stats_label.config(text=self._stats_text())
            except Exception:
                pass

    def clear_history_cache(self):
        global translation_history
        if messagebox.askyesno("Clear History", "Clear all saved history?"):
            translation_history = []
            save_history_cache()
            self.refresh_history_tree()
            self._status("Cleared history")
            try:
                self.stats_label.config(text=self._stats_text())
            except Exception:
                pass

    # -------------------------
    # Helpers (UI, TTS, STT, quiz helper)
    # -------------------------
    def _status(self, msg):
        self.status_var.set(msg)

    def _add_hover(self, widget, on_color="#dfe7ff", off_color=None):
        try:
            if off_color is None:
                off_color = widget.cget("bg")
            def enter(e): widget.config(bg=on_color)
            def leave(e): widget.config(bg=off_color)
            widget.bind("<Enter>", enter)
            widget.bind("<Leave>", leave)
        except Exception:
            pass

    def open_tab_history(self):
        for child in self.root.winfo_children():
            if isinstance(child, ttk.Notebook):
                try:
                    child.select(1)
                except Exception:
                    pass
                break

    def open_tab_translator(self):
        for child in self.root.winfo_children():
            if isinstance(child, ttk.Notebook):
                try:
                    child.select(0)
                except Exception:
                    pass
                break

    def _stats_text(self):
        return f"Points: {user_points}    Streak: {user_streak}    Saved translations: {len(translation_history)}"

    def apply_theme(self):
        dark = self.dark_mode.get()
        bg = "#1f1f1f" if dark else "#f0faff"
        fg = "white" if dark else "black"
        entry_bg = "#2e2e2e" if dark else "white"

        try:
            self.root.configure(bg=bg)
        except Exception:
            pass

        for child in self.root.winfo_children():
            if isinstance(child, ttk.Notebook):
                for tab in child.winfo_children():
                    try:
                        tab.configure(bg=bg)
                        for w in tab.winfo_children():
                            try:
                                if isinstance(w, tk.Frame) or isinstance(w, tk.Label) or isinstance(w, tk.Text) or isinstance(w, tk.LabelFrame):
                                    w.configure(bg=bg if not isinstance(w, tk.LabelFrame) else "white")
                                if isinstance(w, tk.Frame):
                                    for sub in w.winfo_children():
                                        try:
                                            if isinstance(sub, tk.Text):
                                                sub.configure(fg=fg, bg=entry_bg)
                                        except Exception:
                                            pass
                                if isinstance(w, tk.Text):
                                    w.configure(fg=fg, bg=entry_bg)
                            except Exception:
                                pass
                    except Exception:
                        pass

        style = ttk.Style(self.root)
        if dark:
            style.configure("Treeview", background="#262626", foreground="white", fieldbackground="#262626")
            style.map("Treeview", background=[("selected", "#3c5a99")])
        else:
            style.configure("Treeview", background="white", foreground="black", fieldbackground="white")
            style.map("Treeview", background=[("selected", "#cfe2ff")])

        try:
            self.input_text.configure(bg=entry_bg, fg=fg)
            self.output_text.configure(bg=entry_bg, fg="lightgreen" if dark else "green")
        except Exception:
            pass

        try:
            if dark:
                self.stats_label.configure(bg="white")
            else:
                self.stats_label.configure(bg="white")
        except Exception:
            pass

        try:
            self.refresh_history_tree()
        except Exception:
            pass

    def save_settings(self):
        try:
            self.pref_lang.set(self.pref_lang_cb.get())
        except Exception:
            pass
        messagebox.showinfo("Settings", "Settings saved.")
        self._status("Settings saved")

    # -------------------------
    # Quiz helper methods
    # -------------------------
    def _update_quiz_mode_widgets(self):
        if self.quiz_mode.get() == "MCQ":
            self.mcq_frame.pack(fill="x", pady=6)
            try:
                self.quiz_answer_entry.pack_forget()
            except Exception:
                pass
        else:
            try:
                self.mcq_frame.pack_forget()
            except Exception:
                pass
            self.quiz_answer_entry.pack(pady=6)

    def _on_mcq_select(self, idx):
        txt = self.mcq_buttons[idx].cget("text")
        self._mcq_selected_text = txt
        for i, b in enumerate(self.mcq_buttons):
            try:
                b.config(relief="raised")
            except Exception:
                pass
        try:
            self.mcq_buttons[idx].config(relief="sunken")
        except Exception:
            pass

    def speak_text(self, text, use_quiz_lang=False):
        if not text:
            return
        try:
            if use_quiz_lang and hasattr(self, "quiz_lang_cb"):
                lang_code = LANGS.get(self.quiz_lang_cb.get(), "hi")
            else:
                lang_code = LANGS.get(self.pref_lang.get(), "en")
            slow = self.tts_slow.get()
            tts = gTTS(text=text, lang=lang_code if lang_code != "auto" else "en", slow=slow)
            fname = "temp_quiz_tts.mp3"
            tts.save(fname)
            playsound(fname)
            try:
                os.remove(fname)
            except Exception:
                pass
        except Exception as e:
            print("TTS error:", e)

    

    def speak_expected_answer(self):
        exp = getattr(self, "_current_expected", "")
        if exp:
            self.speak_text(exp, use_quiz_lang=True)
        else:
            messagebox.showinfo("Quiz", "No expected answer available. Press 'New Question'.")

    def record_answer_and_fill(self):
        recognizer = sr.Recognizer()
        try:
            with sr.Microphone() as source:
                self._status("Listening for answer... speak now")
                audio = recognizer.listen(source, timeout=6, phrase_time_limit=8)
                try:
                    text = recognizer.recognize_google(audio)
                    self._status("Captured answer")
                    detected = self._detect_language_simple(text)
                    filled = False
                    for i, b in enumerate(self.mcq_buttons):
                        opt = b.cget("text")
                        if opt and self._similar_norm(opt, text):
                            self._on_mcq_select(i)
                            filled = True
                            break
                    if not filled:
                        try:
                            self.quiz_answer_entry.delete(0, tk.END)
                            self.quiz_answer_entry.insert(0, text)
                        except Exception:
                            pass
                    self.quiz_result_label.config(text=f"Captured: {text} (detected: {detected})", fg="blue")
                except sr.UnknownValueError:
                    self._status("Could not understand")
                    messagebox.showinfo("STT", "Could not understand audio")
                except sr.RequestError:
                    self._status("Network error")
                    messagebox.showerror("STT", "Network error during speech recognition")
        except Exception as e:
            messagebox.showerror("Microphone Error", str(e))
            self._status("Microphone error")

    def _similar_norm(self, a, b):
        def n(s): return "".join(ch for ch in s.lower().strip() if ch.isalnum() or ch.isspace())
        na, nb = n(a), n(b)
        return na == nb or na in nb or nb in na

    def _detect_language_simple(self, text):
        try:
            from langdetect import detect
            code = detect(text)
            return code
        except Exception:
            for ch in text:
                if '\u0900' <= ch <= '\u097F':
                    return "hi"
            return "en"

    def _load_quiz_progress(self):
        self._quiz_progress = {"points": 0, "level": 1}
        try:
            if os.path.exists(QUIZ_PROGRESS_FILE):
                with open(QUIZ_PROGRESS_FILE, "r", encoding="utf-8") as f:
                    self._quiz_progress = json.load(f)
        except Exception:
            self._quiz_progress = {"points": 0, "level": 1}
        global user_points, user_streak
        user_points = self._quiz_progress.get("points", 0)
        user_streak = self._quiz_progress.get("streak", 0)

    def _save_quiz_progress(self):
        try:
            self._quiz_progress["points"] = self._quiz_progress.get("points", 0)
            global user_streak
            self._quiz_progress["streak"] = user_streak
            with open(QUIZ_PROGRESS_FILE, "w", encoding="utf-8") as f:
                json.dump(self._quiz_progress, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print("Could not save quiz progress:", e)

    def _award_badge_for_level(self, level):
        if level < 3:
            badge = "Novice"
        elif level < 5:
            badge = "Learner"
        elif level < 8:
            badge = "Skilled"
        else:
            badge = "Master"
        self._quiz_progress["badge"] = badge
        self._save_quiz_progress()
        return badge


# -------------------------
# Run App (with splash)
# -------------------------
if __name__ == "__main__":
    root = tk.Tk()
    root.withdraw()

    def start_app():
        root.deiconify()
        app = TransFlipApp(root)

    show_splash(root, start_app)
    root.mainloop()
